In [1]:
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from random import sample
import csv
# Imports from the func.py file
from func import *
import random

In [2]:
# Reads the graph from the graphml file.
G = nx.read_graphml('../data/graph0.graphml', node_type=int)

In [42]:
# Draws the graph
nx.draw(G, node_size=0, width=.1, alpha=1)

Error in callback <function flush_figures at 0x0000010CAA663670> (for post_execute):


KeyboardInterrupt: 

In [3]:
# Sets up Julia so that we can run Julia files.
from julia import Julia
import julia
# Make sure to change this to your own julia file path.
julia.install(julia=r"C:\Users\Daniel\AppData\Local\Programs\Julia-1.7.2\bin\julia.exe")
julia.Julia(runtime=r"C:\Users\Daniel\AppData\Local\Programs\Julia-1.7.2\bin\julia.exe")
from julia import Main

In [4]:
# Some pre existing measures from the original jupyter notebook to compare local flow betweenness to.
# Current flow betweenness has been removed since it is not implemented for directed graphs.
shortest_path_betweenness = nx.edge_betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

In [5]:
# Includes the Julia files.
Main.include("../julia/local_flow_betweenness.jl");

In [6]:
# Calculates the local flow betweenness for this graph.
# local_flow_betweenness.jl has been modified to work on directed graphs.
local50_betweenness = Main.local_flow_betweenness(list(G.nodes()), list(G.edges()), locality_index=.5)
local10_betweenness = Main.local_flow_betweenness(list(G.nodes()), list(G.edges()), locality_index=.1)
local02_betweenness = Main.local_flow_betweenness(list(G.nodes()), list(G.edges()), locality_index=.02)

In [7]:
perc = .25
reduced_weight = .1

In [8]:
# Uses the functions included in the func.py file to create a weighted adjacency matrix to find the best nodes
# to perform interventions on.
A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)

In [9]:
from collections import Counter
vaccination_methods = [("Shortest Path Betweenness",A_sp), ("Local50 Betweenness",A_lf50), ("Local10 Betweenness",A_lf10), 
                       ("Local02 Betweenness",A_lf02)]

node_ranking = {}

for name, v in vaccination_methods:
        
    v = v.A
    counter = Counter()
    
    ranking = []
    
    for i in range(len(v)):
            
        ranking.append((sum(v[i]), i))
    
    ranking.sort()
    ranking.reverse()
    
    count = 0
    
    print(name)
    for rank, node in ranking:
        if rank > 1.0:
            count+=1
            
    print(count)
#     print(ranking)
        
    # Turns the score, node pairs into just nodes.
    nodes = []
    
    for rank in ranking:
        nodes.append(rank[1])
        
    print(ranking[:42])
        
    node_ranking[name] = nodes
    
random.seed(1)

node_list = list(G.nodes)
random.shuffle(node_list)

node_ranking["Random"] = node_list

Shortest Path Betweenness
42
[(486.60000000000014, 449), (486.60000000000014, 380), (486.60000000000014, 279), (477.60000000000036, 401), (455.30000000000035, 372), (425.4000000000009, 192), (423.6000000000006, 478), (422.7000000000006, 438), (422.7000000000006, 397), (421.80000000000064, 37), (421.80000000000064, 11), (404.1000000000004, 381), (403.1000000000004, 414), (394.60000000000014, 436), (394.5000000000001, 312), (379.5000000000005, 308), (370.50000000000074, 253), (370.50000000000074, 222), (370.50000000000074, 151), (370.50000000000074, 108), (370.50000000000074, 90), (356.0000000000006, 114), (356.0000000000006, 105), (355.30000000000064, 82), (353.6000000000006, 216), (353.6000000000006, 94), (345.00000000000034, 228), (341.3000000000002, 455), (340.4000000000002, 292), (60.60000000000047, 416), (57.3000000000005, 86), (10.299999999999999, 239), (8.5, 142), (8.3, 426), (8.3, 177), (5.1, 245), (4.0, 325), (3.0, 181), (2.100000000000001, 0), (2.1, 333), (2.1, 56), (1.2000000

In [10]:
print(G.nodes)
print(G.edges)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [11]:
adjacency_list = {}

for node in G.nodes:
    adjacency_list[node] = []

for source, dest in G.edges:
    adjacency_list[source].append(dest)

important_nodes = []

for node in adjacency_list:
    if len(adjacency_list[node]) > 0:
        important_nodes.append(node)

print(adjacency_list[449])
print(important_nodes)

for n in node_ranking['Shortest Path Betweenness']:
    count = 0
    for node in adjacency_list:
        if n in adjacency_list[node]:
            count+=1
            
    print(f"{n}: {count}")     

[]
[1, 12, 20, 38, 57, 83, 87, 91, 95, 106, 109, 115, 143, 152, 178, 182, 193, 217, 223, 229, 240, 246, 254, 280, 293, 309, 313, 326, 334, 373, 381, 382, 398, 402, 415, 417, 427, 437, 439, 450, 456, 479]
449: 29
380: 31
279: 31
401: 31
372: 31
192: 31
478: 29
438: 28
397: 31
37: 30
11: 29
381: 23
414: 31
436: 31
312: 31
308: 31
253: 32
222: 31
151: 31
108: 31
90: 31
114: 31
105: 31
82: 31
216: 29
94: 31
228: 29
455: 31
292: 31
416: 29
86: 30
239: 31
142: 31
426: 31
177: 28
245: 31
325: 31
181: 31
0: 0
333: 31
56: 28
19: 31
500: 31
499: 31
498: 31
497: 31
496: 28
495: 31
494: 29
493: 31
492: 26
491: 29
490: 31
489: 31
488: 26
487: 34
486: 29
485: 31
484: 31
483: 26
482: 31
481: 29
480: 26
479: 28
477: 31
476: 31
475: 31
474: 29
473: 31
472: 29
471: 28
470: 31
469: 31
468: 31
467: 29
466: 31
465: 26
464: 31
463: 31
462: 31
461: 31
460: 28
459: 31
458: 31
457: 29
456: 30
454: 31
453: 28
452: 26
451: 31
450: 25
448: 31
447: 29
446: 29
445: 31
444: 31
443: 31
442: 31
441: 32
440: 29
439: 30

In [12]:
outgoing_edges = []

for node in adjacency_list:
    outgoing_edges.append(((len(adjacency_list[node]), node)))

outgoing_edges.sort()
outgoing_edges.reverse()

print(outgoing_edges)

outgoing_edges_ranking = []

for node in outgoing_edges:
    outgoing_edges_ranking.append(node[1])
    
print(outgoing_edges_ranking)

node_ranking["Outgoing Edges"] = outgoing_edges_ranking

[(500, 479), (500, 450), (500, 439), (500, 402), (500, 398), (500, 381), (500, 309), (500, 280), (500, 254), (500, 223), (500, 193), (500, 152), (500, 109), (500, 91), (500, 87), (500, 38), (500, 12), (497, 417), (492, 83), (490, 115), (490, 106), (485, 382), (484, 415), (484, 373), (484, 217), (484, 95), (426, 437), (425, 313), (425, 229), (424, 456), (424, 293), (12, 240), (12, 143), (11, 1), (10, 427), (10, 178), (5, 246), (3, 326), (2, 334), (2, 182), (2, 57), (2, 20), (0, 501), (0, 500), (0, 499), (0, 498), (0, 497), (0, 496), (0, 495), (0, 494), (0, 493), (0, 492), (0, 491), (0, 490), (0, 489), (0, 488), (0, 487), (0, 486), (0, 485), (0, 484), (0, 483), (0, 482), (0, 481), (0, 480), (0, 478), (0, 477), (0, 476), (0, 475), (0, 474), (0, 473), (0, 472), (0, 471), (0, 470), (0, 469), (0, 468), (0, 467), (0, 466), (0, 465), (0, 464), (0, 463), (0, 462), (0, 461), (0, 460), (0, 459), (0, 458), (0, 457), (0, 455), (0, 454), (0, 453), (0, 452), (0, 451), (0, 449), (0, 448), (0, 447), (0

In [13]:
incoming_edges = []

# Counts in degree.
counter = Counter()
for node in adjacency_list:
    counter[node] = 0
    for neighbour in adjacency_list[node]:
        counter[neighbour]+=1

# Converts count into list of tuples in format (in degree, node)
incoming_edges = []

for node in counter:
    incoming_edges.append((counter[node], node))

incoming_edges.sort()
incoming_edges.reverse()

incoming_edges_ranking = []

for node in incoming_edges:
    incoming_edges_ranking.append(node[1])
    
node_ranking["Incoming Edges"] = incoming_edges_ranking

In [14]:
# # Random selection version

# # Takes the sets of infected and vaccinated nodes and returns the id of a new node to infect.
# # Returns None if there is no new node to infect.
# def findNewInfection(infected: set, vaccinated: set) -> int:
    
#     # List of infectable nodes to select our next infection from.
#     infectable = []

#     # Go through each infected node and find their neighbours to find nodes to spread to.
#     for node in infected:
#         for neighbour in adjacency_list[node]:
#             if neighbour not in vaccinated and neighbour not in infected:
#                 infectable.append(neighbour)
    
# #     print(len(infectable))
    
#     # If there are no nodes to infect, return None.
#     if len(infectable) == 0:
#         return None
    
#     # Otherwise, pick a new node randomly from the list of infectable nodes.
#     return infectable[random.randrange(0, len(infectable))]

# # Takes the list of node rankings, and the number of vaccinated nodes, returns the id of a new node to vaccinate.
# def findNewVaccination(nr, num_vaccinated) -> int:
    
#     return nr[num_vaccinated]
    
# simulation_num = 10  

# # Try each vaccination method.
# for name in node_ranking:
    
#     # Keep track of the number of infected nodes with each type of vaccination method. 
#     num_infected_nodes = 0
    
#     # Set the node ranking method using the name from node_ranking.
#     nr = node_ranking[name]
    
#     # Simulate the infection with all nodes as starting nodes.
#     for infection_source in G.nodes:
        
#         # Run simulation_num simulations.
#         for i in range(simulation_num):
            
#             # Set random seed to create reproducable results.
#             random.seed(i)

#             # List containing the current nodes which are infected.
#             infected = set()

#             # List containing the vaccinated nodes, so we can pick which ones to intervene on.
#             vaccinated = set()
            
#             # Number of nodes vaccinated to keep track of which node to vaccinate next in the node ranking list.
#             num_vaccinated = 0
            
#             # The source computer starts off as infected.
#             infected.add(infection_source)
#             num_infected_nodes+=1
            
#             # While the ransomware has spread in the last round, keep going.
#             while True:
                
#                 new_infection = findNewInfection(infected, vaccinated)
                
#                 # If there are no new nodes to infect, end the simulation.
#                 if new_infection is None:
#                     break
                                
#                 # Otherwise, add it to infected.
#                 infected.add(new_infection)
#                 num_infected_nodes+=1
                
#                 new_vaccination = findNewVaccination(nr, num_vaccinated)
                
#                 # Vaccinates the selected node.
#                 vaccinated.add(new_vaccination)
        
# #             print(f"{name}: {i+1} out of {simulation_num} complete.")
                
#     print(name, num_infected_nodes)

In [27]:
# Important nodes version.

# Takes the sets of infected and vaccinated nodes and returns the id of a new node to infect.
# Returns None if there is no new node to infect.
def findNewInfection(infected: set, vaccinated: set) -> int:
    
    # List of infectable nodes to select our next infection from.
    infectable = []

    # Go through each infected node and find their neighbours to find nodes to spread to.
    for node in infected:
        for neighbour in adjacency_list[node]:
            if neighbour not in vaccinated and neighbour not in infected:
                infectable.append(neighbour)
        
    # If there are no nodes to infect, return None.
    if len(infectable) == 0:
        return None
    
    # Otherwise, pick a new node randomly from the list of infectable nodes.
    return infectable[random.randrange(0, len(infectable))]

# Takes the list of node rankings, and the number of vaccinated nodes, returns the id of a new node to vaccinate.
def findNewVaccination(nr, num_vaccinated) -> int:
    
    return num_vaccinated+1, nr[num_vaccinated]
    
simulation_num = 1

simulations_per_method = simulation_num*len(important_nodes)

# Try each vaccination method.
for name in node_ranking:
    
    # Set the node ranking method using the name from node_ranking.
    nr = node_ranking[name]
    
    # Keep track of the number of infected nodes with each type of vaccination method. 
    num_infected_nodes = 0
    
    # Simulate the infection with all nodes as starting nodes.
    for infection_source in important_nodes:
        
        # Run simulation_num simulations.
        for i in range(simulation_num):
            
            # Set random seed to create reproducable results.
            random.seed(i)

            # List containing the current nodes which are infected.
            infected = set()

            # List containing the vaccinated nodes, so we can pick which ones to intervene on.
            vaccinated = set()
            
            # Number of nodes vaccinated to keep track of which node to vaccinate next in the node ranking list.
            num_vaccinated = 0
            
            # The source computer starts off as infected.
            infected.add(infection_source)
            num_infected_nodes+=1
            
            # While the ransomware has spread in the last round, keep going.
            while True:
                
                new_infection = findNewInfection(infected, vaccinated)
                
                # If there are no new nodes to infect, end the simulation.
                if new_infection is None:
                    break
                                
                # Otherwise, add it to infected.
                infected.add(new_infection)
                
                num_infected_nodes+=1
                
                num_vaccinated, new_vaccination = findNewVaccination(nr, num_vaccinated)
                print(new_vaccination) 
                
                # Vaccinates the selected node.
                vaccinated.add(new_vaccination)
                if new_vaccination in infected:
                    infected.remove(new_vaccination)
                    
#                 print(infected)
#                 print(vaccinated)
                
#                 print(num_vaccinated)
                
#             print(f"{name}: {i+1} out of {simulation_num} complete.")

            sys.exit(0)
      
    print(f"{name} had an average of {num_infected_nodes/simulations_per_method} infections per simulation.")

449
380
279
401
372
192
478
438
397
37
11
381
414
436
312
308
253
222
151
108
90
114
105
82
216
94
228
455
292
416
86
239
142
426
177
245
325
181
0
333
56
19
500
499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
454
453
452
451
450
448
447
446
445
444
443
442
441
440
439
437
435
434
433
432
431
430
429
428
427
425
424
423
422
421
420
419
418
417
415
413
412
411
410
409
408
407
406
405
404
403
402
400
399
398
396
395
394
393
392
391
390
389
388
387
386
385
384
383
382
379
378
377
376
375
374
373
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
332
331
330
329
328
327
326
324
323
322
321
320
319
318
317
316
315
314
313
311
310
309
307
306
305
304
303
302
301
300
299
298
297
296
295
294
293
291
290
289
288
287
286
285
284
283
282
281
280
278
277
276
275
274
273
272
27

NameError: name 'sys' is not defined

In [26]:
print(node_ranking["Shortest Path Betweenness"])
print(node_ranking["Outgoing Edges"])

[449, 380, 279, 401, 372, 192, 478, 438, 397, 37, 11, 381, 414, 436, 312, 308, 253, 222, 151, 108, 90, 114, 105, 82, 216, 94, 228, 455, 292, 416, 86, 239, 142, 426, 177, 245, 325, 181, 0, 333, 56, 19, 500, 499, 498, 497, 496, 495, 494, 493, 492, 491, 490, 489, 488, 487, 486, 485, 484, 483, 482, 481, 480, 479, 477, 476, 475, 474, 473, 472, 471, 470, 469, 468, 467, 466, 465, 464, 463, 462, 461, 460, 459, 458, 457, 456, 454, 453, 452, 451, 450, 448, 447, 446, 445, 444, 443, 442, 441, 440, 439, 437, 435, 434, 433, 432, 431, 430, 429, 428, 427, 425, 424, 423, 422, 421, 420, 419, 418, 417, 415, 413, 412, 411, 410, 409, 408, 407, 406, 405, 404, 403, 402, 400, 399, 398, 396, 395, 394, 393, 392, 391, 390, 389, 388, 387, 386, 385, 384, 383, 382, 379, 378, 377, 376, 375, 374, 373, 371, 370, 369, 368, 367, 366, 365, 364, 363, 362, 361, 360, 359, 358, 357, 356, 355, 354, 353, 352, 351, 350, 349, 348, 347, 346, 345, 344, 343, 342, 341, 340, 339, 338, 337, 336, 335, 334, 332, 331, 330, 329, 328, 327,

In [25]:
print(node_ranking)

{'Shortest Path Betweenness': [449, 380, 279, 401, 372, 192, 478, 438, 397, 37, 11, 381, 414, 436, 312, 308, 253, 222, 151, 108, 90, 114, 105, 82, 216, 94, 228, 455, 292, 416, 86, 239, 142, 426, 177, 245, 325, 181, 0, 333, 56, 19, 500, 499, 498, 497, 496, 495, 494, 493, 492, 491, 490, 489, 488, 487, 486, 485, 484, 483, 482, 481, 480, 479, 477, 476, 475, 474, 473, 472, 471, 470, 469, 468, 467, 466, 465, 464, 463, 462, 461, 460, 459, 458, 457, 456, 454, 453, 452, 451, 450, 448, 447, 446, 445, 444, 443, 442, 441, 440, 439, 437, 435, 434, 433, 432, 431, 430, 429, 428, 427, 425, 424, 423, 422, 421, 420, 419, 418, 417, 415, 413, 412, 411, 410, 409, 408, 407, 406, 405, 404, 403, 402, 400, 399, 398, 396, 395, 394, 393, 392, 391, 390, 389, 388, 387, 386, 385, 384, 383, 382, 379, 378, 377, 376, 375, 374, 373, 371, 370, 369, 368, 367, 366, 365, 364, 363, 362, 361, 360, 359, 358, 357, 356, 355, 354, 353, 352, 351, 350, 349, 348, 347, 346, 345, 344, 343, 342, 341, 340, 339, 338, 337, 336, 335, 334,

In [30]:
for name in node_ranking:
    print(name, node_ranking[name][:42])

Shortest Path Betweenness [449, 380, 279, 401, 372, 192, 478, 438, 397, 37, 11, 381, 414, 436, 312, 308, 253, 222, 151, 108, 90, 114, 105, 82, 216, 94, 228, 455, 292, 416, 86, 239, 142, 426, 177, 245, 325, 181, 0, 333, 56, 19]
Local50 Betweenness [192, 438, 11, 372, 279, 151, 86, 308, 416, 397, 401, 449, 478, 37, 82, 108, 380, 253, 222, 90, 414, 105, 381, 216, 114, 94, 455, 312, 436, 228, 292, 142, 239, 426, 0, 177, 245, 325, 56, 19, 333, 181]
Local10 Betweenness [438, 416, 478, 401, 449, 397, 372, 380, 279, 308, 192, 414, 381, 222, 253, 151, 86, 82, 108, 455, 216, 436, 11, 90, 312, 37, 94, 105, 114, 228, 292, 142, 239, 426, 0, 177, 245, 325, 333, 181, 56, 19]
Local02 Betweenness [192, 11, 438, 372, 151, 279, 86, 308, 416, 397, 37, 401, 108, 82, 478, 449, 222, 380, 253, 105, 90, 114, 414, 94, 381, 216, 455, 312, 436, 228, 292, 426, 239, 142, 177, 245, 325, 0, 333, 181, 56, 19]
Random [161, 270, 313, 307, 449, 406, 194, 309, 382, 215, 395, 269, 169, 417, 64, 245, 227, 322, 78, 366, 186,